In [1]:
import os
import datetime
import arxiv
from dotenv import load_dotenv
import autogen
from autogen import UserProxyAgent, AssistantAgent, GroupChat, GroupChatManager

In [2]:
# Step 1: Configure GPT-4 / GPT-3.5 directly with API key
gpt4_config = {
    "cache_seed": 42,
    "temperature": 0,
    "config_list": [
        {
            "model": "gpt-3.5-turbo",  # or use "gpt-4"
            "api_key": os.getenv("OPENAI_API_KEY"),
        }
    ],
    "timeout": 120,
}


In [3]:
import sys
print(sys.executable)

e:\conda_envs\autogenenv\python.exe


In [4]:
# Step 2: Define agents
user_proxy = UserProxyAgent(
    name="Admin",
    system_message="Human admin. You approve plans.",
    code_execution_config=False
)

engineer = AssistantAgent(
    name="Engineer",
    llm_config=gpt4_config,
    system_message="""Engineer who writes Python code to scrape and format arXiv data.
Only give complete runnable code in markdown blocks."""
)

scientist = AssistantAgent(
    name="Scientist",
    llm_config=gpt4_config,
    system_message="""Scientist who categorizes arXiv papers based on their titles and summaries. Do not write code."""
)

executor = UserProxyAgent(
    name="Executor",
    system_message="Runs code from Engineer and reports results.",
    human_input_mode="NEVER",
    code_execution_config={
        "last_n_messages": 3,
        "work_dir": "paper",
        "use_docker": False
    }
)


planner = AssistantAgent(
    name="Planner",
    llm_config=gpt4_config,
    system_message="""Planner who breaks down the task.
Assigns steps to Engineer and Scientist. Plan must be detailed and actionable."""
)

In [5]:
# Step 3: Define group chat
groupchat = GroupChat(
    agents=[user_proxy, planner, engineer, scientist, executor],
    messages=[],
    max_round=20
)

manager = GroupChatManager(groupchat=groupchat, llm_config=gpt4_config)

In [6]:
# Step 4: Start chat
user_proxy.initiate_chat(
    manager,
    message="""
Find arXiv papers from the last 7 days related to LLM applications and generate a markdown table categorized by domain. Include title, authors, summary, and link for each paper.
"""
)

Admin (to chat_manager):


Find arXiv papers from the last 7 days related to LLM applications and generate a markdown table categorized by domain. Include title, authors, summary, and link for each paper.


--------------------------------------------------------------------------------

Next speaker: Planner

Planner (to chat_manager):

**Task Breakdown:**

**Engineer:**
1. Develop a script to scrape arXiv for papers related to LLM applications from the last 7 days.
2. Extract relevant information such as title, authors, summary, and link for each paper.
3. Organize the extracted data into a structured format for further processing.

**Scientist:**
1. Review the extracted papers to categorize them based on domains related to LLM applications.
2. Write a brief summary for each paper based on its content and relevance to the domain.
3. Create a markdown table template with columns for title, authors, summary, and link categorized by domain.

**Collaborative Task:**
1. Engineer to integr

ChatResult(chat_id=None, chat_history=[{'content': '\nFind arXiv papers from the last 7 days related to LLM applications and generate a markdown table categorized by domain. Include title, authors, summary, and link for each paper.\n', 'role': 'assistant', 'name': 'Admin'}, {'content': "**Task Breakdown:**\n\n**Engineer:**\n1. Develop a script to scrape arXiv for papers related to LLM applications from the last 7 days.\n2. Extract relevant information such as title, authors, summary, and link for each paper.\n3. Organize the extracted data into a structured format for further processing.\n\n**Scientist:**\n1. Review the extracted papers to categorize them based on domains related to LLM applications.\n2. Write a brief summary for each paper based on its content and relevance to the domain.\n3. Create a markdown table template with columns for title, authors, summary, and link categorized by domain.\n\n**Collaborative Task:**\n1. Engineer to integrate the structured data into the markdo